<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 800; font-size: 2.5em; color: white; text-shadow: 2px 2px 4px #000;">✨ 2 CLASS CLASSIFICATION, SAVING THE MODEL , IMPLEMENTING GRADCAM✨</span>
</div>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 800; font-size: 2.5em; color: white; text-shadow: 2px 2px 4px #000;">✨ LOADING THE SPLIT DATA ARRAYS ✨</span>
</div>

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">📁 reminder to change the folder path to your numpy array folder 📁</span>
</div>

In [ ]:
import numpy as np
import os

# Define the folder path
folder_path = r"D:\PYTHONIG\newwindow\numpy\NEWDATA\ARRAYS\mel"

# Load the numpy files into the respective arrays
eeg_fold_1 = np.load(os.path.join(folder_path, 'mel_data_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'mel_labels_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'mel_patients_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'mel_data_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'mel_labels_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'mel_patients_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'mel_data_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'mel_labels_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'mel_patients_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'mel_data_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'mel_labels_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'mel_patients_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'mel_data_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'mel_labels_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'mel_patients_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">converting to 2 class</span>
</div>

In [ ]:
for i in range (len(labels_fold_1)):
    if (labels_fold_1[i] == 2):
        labels_fold_1[i] = 1

for i in range (len(labels_fold_2)):
    if (labels_fold_2[i] == 2):
        labels_fold_2[i] = 1

for i in range (len(labels_fold_3)):
    if (labels_fold_3[i] == 2):
        labels_fold_3[i] = 1

for i in range (len(labels_fold_4)):
    if (labels_fold_4[i] == 2):
        labels_fold_4[i] = 1

for i in range (len(labels_fold_5)):
    if (labels_fold_5[i] == 2):
        labels_fold_5[i] = 1

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">data balancer & early stopping</span>
</div>

In [ ]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)


    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)


    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">Result plotting</span>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
    auc,
    f1_score,
    balanced_accuracy_score,
    classification_report,
)
from sklearn.preprocessing import label_binarize

def plot_metrics(labels, predictions, n_classes=2):
    """
    Computes and visualizes classification metrics.

    Args:
        labels (array-like): True class labels.
        predictions (array-like): Probabilities or class predictions.
        n_classes (int): Number of classes (default: 2 for binary classification).
    """
    
    # Convert probability predictions to class labels if needed
    if predictions.ndim > 1:
        predicted_classes = np.argmax(predictions, axis=1)
    else:
        predicted_classes = predictions

    # Compute Confusion Matrix
    cm = confusion_matrix(labels, predicted_classes)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

    # Class-wise Accuracy
    print("\nClass-wise Accuracy:")
    class_accuracies = []
    for d in range(n_classes):
        correct_preds = cm[d][d]
        total_true_samples = sum(cm[d])
        accuracy = correct_preds / total_true_samples if total_true_samples > 0 else 0
        class_accuracies.append(accuracy)
        print(f'Class {d}: {correct_preds}/{total_true_samples} ({accuracy:.2%})')

    # Precision-Recall Curve & AUPRC
    print("\nPrecision-Recall Curve:")
    plt.figure()
    
    if n_classes == 2:
        # Binary classification
        precision, recall, _ = precision_recall_curve(labels, predictions[:, 1])
        auprc = auc(recall, precision)
        plt.plot(recall, precision, label=f'AUPRC = {auprc:.2f}')
        avg_auprc = auprc
    else:
        # Multi-class case
        labels_binarized = label_binarize(labels, classes=np.arange(n_classes))
        auprcs = []
        for class_idx in range(n_classes):
            precision, recall, _ = precision_recall_curve(labels_binarized[:, class_idx], predictions[:, class_idx])
            auprc = auc(recall, precision)
            auprcs.append(auprc)
            plt.plot(recall, precision, label=f'Class {class_idx} (AUPRC = {auprc:.2f})')
        avg_auprc = np.mean(auprcs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

    # ROC Curve & AUC
    print("\nROC Curve:")
    plt.figure()
    
    if n_classes == 2:
        # Binary classification
        fpr, tpr, _ = roc_curve(labels, predictions[:, 1])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    else:
        # Multi-class case
        for class_idx in range(n_classes):
            fpr, tpr, _ = roc_curve(labels_binarized[:, class_idx], predictions[:, class_idx])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'Class {class_idx} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Compute Metrics
    f1 = f1_score(labels, predicted_classes, average='macro')
    balanced_acc = balanced_accuracy_score(labels, predicted_classes)

    print(f"\nMean F1 Score: {f1:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"Average AUPRC: {avg_auprc:.4f}")

    print("\nClassification Report:")
    print(classification_report(labels, predicted_classes))

    return {
        "confusion_matrix": cm,
        "class_wise_accuracy": class_accuracies,
        "mean_f1_score": f1,
        "balanced_accuracy": balanced_acc,
    }


<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">MODEL GOES HERE</span>
</div>

In [4]:
import torch
import torch.nn as nn
import numpy as np

def patchify(data, n_patches):
    n, c, h, w = data.shape
    patches = torch.zeros(n, n_patches**2, (c*h*w) // (n_patches**2))
    patch_size = h // n_patches
    for idx, d in enumerate(data):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = d[:, i*patch_size: (i+1)*patch_size, j*patch_size: (j+1) * patch_size]
                patches[idx, i*n_patches + j] = patch.flatten()
    return patches

def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result

class vit(nn.Module):
    def __init__(self, input_size, n_patches, hidden_size, num_classes):
        super().__init__()
        self.chw = input_size
        self.n_patches = n_patches
        self.patch_size = ((self.chw[1] // n_patches) ** 2) * self.chw[0]
        self.linear_mapper = nn.Linear(self.patch_size, hidden_size)
        self.class_token = nn.Parameter(torch.rand(1, hidden_size))
        self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1, hidden_size)))
        self.pos_embed.requires_grad = True
        self.layer_norm = nn.LayerNorm(hidden_size)

        # Transformer encoder layers
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=4, batch_first=True)
            for _ in range(2)
        ])
        self.classifier = nn.Sequential(nn.Linear(hidden_size, num_classes))

        # For saliency maps
        self.gradients = None
        self.activations = None
        self._register_hook = False

    def save_gradient(self, grad):
        self.gradients = grad

    def forward(self, x):
        patches = patchify(x, self.n_patches)
        
        x = self.linear_mapper(patches)
        x = torch.stack([torch.vstack((self.class_token, x[i])) for i in range(len(x))])
        pos_embed = self.pos_embed.repeat(len(x), 1, 1)
        x = x + pos_embed
        x = self.layer_norm(x)

        # Pass through transformer encoder layers
        for idx, layer in enumerate(self.encoder_layers):
            x = layer(x)
            if idx == len(self.encoder_layers) - 1 and not self._register_hook:
                x.register_hook(self.save_gradient)  # Register hook only once
                self._register_hook = True
            self.activations = x  # Save the latest activations

        x = x[:, 0]  # Select CLS token for classification
        x = self.classifier(x)
        return x

    def get_activations_gradient(self):
        return self.gradients

    def get_activations(self):
        return self.activations

    def set_hook(self, hook_status):
        self._register_hook = hook_status

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">Test with demo data & model Summary</span>
</div>

In [9]:
model  = vit(input_size=(20,100,100),n_patches=10,hidden_size=512,num_classes=2)# declare model here
randomdata = torch.randn((1,20,100,100))
output = model(randomdata)
output.shape

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_15684\1373862186.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1, hidden_size)))


torch.Size([1, 2])

In [7]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                       Param #
vit                                                          52,224
├─Linear: 1-1                                                1,024,512
├─LayerNorm: 1-2                                             1,024
├─ModuleList: 1-3                                            --
│    └─TransformerEncoderLayer: 2-1                          --
│    │    └─MultiheadAttention: 3-1                          1,050,624
│    │    └─Linear: 3-2                                      1,050,624
│    │    └─Dropout: 3-3                                     --
│    │    └─Linear: 3-4                                      1,049,088
│    │    └─LayerNorm: 3-5                                   1,024
│    │    └─LayerNorm: 3-6                                   1,024
│    │    └─Dropout: 3-7                                     --
│    │    └─Dropout: 3-8                                     --
│    └─TransformerEncoderLayer: 2-2                       

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">Training Code, TRAINING ONLY ONE FOLD, TO SAVE MODEL</span>
</div>

In [ ]:
import pandas as pd
import os
import datetime


# Fold indices and other configurations
num_folds = 1 # ONLY ONE FOLD
fold_indices = [0,1,2,3,4]
val_fold_indices = np.roll(fold_indices, 1)

test_folds_chosen = []
val_folds_chosen = []
fold_confusion_matrices = []  # To store confusion matrices for each fold
fold_accuracies = []  # To store balanced accuracy for each fold
fold_auprcs = []  # To store AUPRC for each fold

# Model and training configurations
num_classes = 2
learning_rate = 0.0001
epochs = 1
input_shape = (20, 100, 100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, auc




# Training loop for cross-validation
for fold_idx in range(num_folds):
    print(f'Fold No: {fold_idx + 1}')
    
    best_val_acc = 0.0
    
    # Initialize model, loss, and optimizer
    fold_model = vit(input_size=input_shape,n_patches=10,hidden_size=512,num_classes=2)
    fold_model.to(device)
    
    criterion = nn.CrossEntropyLoss(label_smoothing=0.3)
    optimizer = optim.Adam(fold_model.parameters(), lr=learning_rate)
    
    # Split data into train, validation, and test sets
    test_fold = fold_indices[fold_idx]
    val_fold = val_fold_indices[fold_idx]
    train_folds = [fold for fold in fold_indices if fold != test_fold and fold != val_fold]

    train_data = np.concatenate([eeg_folds[j] for j in train_folds])
    train_labels = np.concatenate([labels_folds[j] for j in train_folds])
    train_data = train_data.transpose(0, 3, 1, 2)  # Transpose to match PyTorch input format

    test_folds_chosen.append(test_fold)
    val_folds_chosen.append(val_fold)
    
    early_stopping = EarlyStopping(patience=10)
    
    # Training loop
    for epoch in range(epochs):
        balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=1)

        train_dataset = TensorDataset(
            torch.tensor(balanced_train_data, dtype=torch.float32).to(device),
            torch.tensor(balanced_train_labels, dtype=torch.long).to(device)
        )
        train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
        
        fold_model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = fold_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = balanced_accuracy_score(all_labels, all_preds)
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        val_data = eeg_folds[val_fold].transpose(0, 3, 1, 2)
        val_labels = labels_folds[val_fold]
        val_dataset = TensorDataset(
            torch.tensor(val_data, dtype=torch.float32).to(device),
            torch.tensor(val_labels, dtype=torch.long).to(device)
        )
        val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

        fold_model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels_list = []

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = fold_model(val_inputs)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()

                _, val_batch_preds = torch.max(val_outputs, 1)
                val_preds.extend(val_batch_preds.cpu().numpy())
                val_labels_list.extend(val_labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = balanced_accuracy_score(val_labels_list, val_preds)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

        early_stopping(val_loss, fold_model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    # Load the best model
    early_stopping.load_best_model(fold_model)
    
    if val_acc > best_val_acc:
        best_val_acc =  val_acc
        model_save_path = fr"D:\PYTHONIG\newwindow\NEW DATA RUNS\Vit_gradcam\stft_model_fold_{i+1}_1P.pth"
        torch.save({ 'MEL_model_state_dict': model.state_dict(), 'MEL_optimizer_state_dict': optimizer.state_dict(), 'epoch': epoch}, model_save_path)
        print(f'Model for fold {i+1} saved at epoch{epoch +1}')

    # Test loop
    test_data = eeg_folds[test_fold].transpose(0, 3, 1, 2)
    test_labels = labels_folds[test_fold]
    test_dataset = TensorDataset(
        torch.tensor(test_data, dtype=torch.float32).to(device),
        torch.tensor(test_labels, dtype=torch.long).to(device)
    )
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    fold_model.eval()
    test_probs = []
    test_preds = []
    test_labels_list = []

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = fold_model(test_inputs)
            probabilities = torch.softmax(test_outputs, dim=1)
            test_probs.extend(probabilities.cpu().numpy())
            _, preds = torch.max(probabilities, 1)
            test_preds.extend(preds.cpu().numpy())
            test_labels_list.extend(test_labels.cpu().numpy())

    # Compute metrics
    test_acc = balanced_accuracy_score(test_labels_list, test_preds)
    fold_accuracies.append(test_acc)

    cm = confusion_matrix(test_labels_list, test_preds)
    fold_confusion_matrices.append(cm)

    test_labels_binarized = test_labels_list
    test_auprcs = []

    precision, recall, _ = precision_recall_curve(test_labels_list, np.array(test_probs)[:, 1])
    auprc = auc(recall, precision)
    test_auprcs.append(auprc)

    mean_test_auprc = np.mean(test_auprcs)
    fold_auprcs.append(mean_test_auprc)

    print(f'Test Fold {fold_idx + 1}, Mean AUPRC: {mean_test_auprc:.4f}, Balanced Accuracy: {test_acc:.4f}')

    # Use the plot_metrics function to visualize metrics
    plot_metrics(np.array(test_labels_list), np.array(test_probs), n_classes=num_classes)

# Final metrics across all folds
average_auprc = np.mean(fold_auprcs)
mean_accuracy = np.mean(fold_accuracies)
print(f'Accuracy for each fold: {fold_accuracies}')
print(f'AUPRC for each fold: {fold_auprcs}')
print(f'Average AUPRC across all folds: {average_auprc}')
print(f'Average Balanced Accuracy across all folds: {mean_accuracy}')
    


<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">GRADCAM FUNCTIONS</span>
</div>

In [ ]:
import matplotlib.pyplot as plt

def load_model(model_path, model_class, device):
    model = model_class((20, 100, 100), 25, 512).to(device)  
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['MEL_model_state_dict'])
    model.eval()
    return model



def depatchify(grad_cam, n_patches, image_size):
    """
    Reverse the patchification process to reconstruct the full image Grad-CAM from patch-level scores.
    Args:
        grad_cam: Grad-CAM heatmap of shape [n_patches^2].
        n_patches: Number of patches along each dimension.
        image_size: Tuple (H, W) of the original image dimensions.
    Returns:
        heatmap: Full image heatmap of shape [H, W].
    """
    patch_size = image_size[0] // n_patches
    heatmap = np.zeros(image_size)

    for i in range(n_patches):
        for j in range(n_patches):
            patch_idx = i * n_patches + j
            patch_value = grad_cam[patch_idx]  # Get Grad-CAM value for this patch
            heatmap[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] = patch_value

    return heatmap

In [ ]:
class GradCAM:
    def __init__(self, model):
        self.model = model

    def __call__(self, input_tensor, target_class=None):
        self.model.eval()
        output = self.model(input_tensor)  # Forward pass
        if target_class is None:
            target_class = output.argmax(dim=-1)  # Predicted class for each batch item
            print ('target_class is: ', target_class, target_class.shape)

        # Compute gradients for the target clas s
        loss = output[:, target_class].sum()
        self.model.zero_grad()
        loss.backward()

        # Get activations and gradients
        gradients = self.model.gradients  # Shape: [batch_size, seq_len, hidden_dim]
        activations = self.model.activations  # Shape: [batch_size, seq_len, hidden_dim]
        # print('gradients shape', gradients[:,1:].sum())
        
        weights = torch.mean(gradients, dim=1, keepdim=True)  # Mean over sequence length
        # print('weights:', weights)
        cam = torch.sum(weights * activations, dim=-1).squeeze()  # Weighted sum
        # print('cam', cam)
        
        cam = cam - cam.min()
        cam = cam / cam.max()
        cam=cam[:,1:]
        return cam


In [ ]:
def plot_gradcam(input_image, grad_cam_heatmap,spectrogram_image, alpha=0.5, cmap='inferno'):
    """
    Plot the input spectrogram with Grad-CAM heatmap overlay.
    Args:
        input_image: Input spectrogram of shape [H, W].
        grad_cam_heatmap: Full Grad-CAM heatmap of shape [H, W].
        alpha: Transparency for overlay (0.0 to 1.0).
    """
    Fs = 256  
    n_bins = 100
    T = 20 
    n_time_bins = 100
    frequency_axis = np.linspace(0, Fs/2, n_bins)
    
    time_axis = np.linspace(0, T, n_time_bins)
    

    normalized_spectrogram_image = (spectrogram_image - spectrogram_image.min()) / (spectrogram_image.max() - spectrogram_image.min())
    normalized_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    grad_cam_heatmap = (grad_cam_heatmap - grad_cam_heatmap.min()) / (grad_cam_heatmap.max() - grad_cam_heatmap.min())

    plt.figure(figsize=(10, 5))

    plt.subplot(2, 2, 1)
    plt.imshow(normalized_spectrogram_image, aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    plt.colorbar() 
    
    plt.ylabel('Frequency (Hz)')
    plt.title('EEG Time-Frequency Spectrogram')
    

    plt.subplot(2, 2, 2)
    # plt.imshow(normalized_image, vmin=0, vmax=1, aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    plt.imshow(grad_cam_heatmap,  interpolation='bilinear', alpha=alpha,aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    plt.title("Grad-CAM Visualization")
    plt.colorbar()

    plt.subplot(2, 2, 3)
    plt.imshow(normalized_image, vmin=0, vmax=1, aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    plt.colorbar()
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    
    

    # Plot heatmap overlay
    plt.subplot(2, 2, 4)
    plt.imshow(normalized_image, vmin=0, vmax=1, aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    plt.imshow(grad_cam_heatmap,  interpolation='bilinear', alpha=alpha,aspect='auto', origin='lower', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()],   cmap=cmap)
    
    plt.colorbar()
    
    
    plt.show()


In [ ]:
import torch

def test_model(model, data_loader, device):
    """
    Test the model on the first `num_samples` images and compare predictions with actual labels.
    Args:
        model: Trained model.
        data_loader: DataLoader for the test dataset.
        device: Device ('cpu' or 'cuda').
        num_samples: Number of samples to test (default is 10).
    """
    model.eval()  # Set model to evaluation mode
    actual_labels = []
    predicted_labels = []
    images = []

    with torch.no_grad():
        for inputs,labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)  # Get predicted class
            actual_labels.extend(labels.cpu().tolist())
            predicted_labels.extend(preds.cpu().tolist())
            images.extend(inputs.cpu())


    # Print results for the first `num_samples` images
    print(f"{'Image':<10}{'Actual Label':<15}{'Predicted Label':<15}")
    for i in range(len(labels)):
        print(f"{i+1:<10}{actual_labels[i]:<15}{predicted_labels[i]:<15}")

    return images, actual_labels, predicted_labels


In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(images, actual_labels, predicted_labels):
    """
    Visualize spectrogram images with actual and predicted labels.
    Args:
        images: List of spectrogram images.
        actual_labels: List of actual labels.
        predicted_labels: List of predicted labels.
        num_samples: Number of samples to visualize.
    """
    for i in range(len(actual_labels)):
        plt.figure(figsize=(4, 4))
        plt.imshow(images[i][0], cmap="viridis")  # Assuming single-channel image
        plt.title(f"Actual: {actual_labels[i]}, Predicted: {predicted_labels[i]}")
        plt.axis("off")
        plt.show()


<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">Getting Testing DATA for fold 1</span>
</div>

In [ ]:
test_data = eeg_folds[0]
test_data= np.transpose(test_data, (0,3,1,2))

test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(labels_folds[0], dtype=torch.long).to(device)
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">LOAD SAVED MODEL</span>
</div>

In [ ]:
model = load_model(r"D:\PYTHONIG\newwindow\GRADCAM\20patchsize\model_fold_1_1P.pth", vit, device) 
images, actual_labels, predicted_labels = test_model(model, test_loader, device)

In [ ]:
len(actual_labels),len(images),len(predicted_labels)

In [ ]:
torch.cuda.empty_cache()

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">CLASS 0 PREDICTED 0</span>
</div>

In [ ]:
import numpy as np

device = torch.device('cuda')

Fs = 256  
n_bins = 100
T = 20 
n_time_bins = 100
frequency_axis = np.linspace(0, Fs/2, n_bins)

time_axis = np.linspace(0, T, n_time_bins)

image_size = (100, 100)
n_patches = model.n_patches
patch_size = image_size[0] // n_patches

# Instantiate Grad-CAM
grad_cam = GradCAM(model)
cam = grad_cam(test_data_tensor, target_class=0)  
average_heatmap_class_0 = np.zeros(image_size)
count_class_0 = 0


for idx in range(len(actual_labels)):
    if actual_labels[idx] == predicted_labels[idx]:  
        cam_batch = cam[idx, :].detach().cpu().numpy()  
        heatmap = np.zeros(image_size)
        
        # Reconstruct the heatmap from patches
        for i in range(n_patches):
            for j in range(n_patches):
                patch_idx = i * n_patches + j
                patch_value = cam_batch[patch_idx]  # Get Grad-CAM value for this patch
                heatmap[
                    i * patch_size: (i + 1) * patch_size,
                    j * patch_size: (j + 1) * patch_size
                ] = patch_value
        
        # Accumulate heatmaps for the corresponding class
        if actual_labels[idx] == 0:  # True prediction for class 0
            average_heatmap_class_0 += heatmap
            count_class_0 += 1

# Compute average heatmaps
if count_class_0 > 0:
    average_heatmap_class_0 /= count_class_0


# Normalize the average heatmaps (optional, for visualization)
average_heatmap_class_0 -= average_heatmap_class_0.min()
average_heatmap_class_0 /= average_heatmap_class_0.max()



# Final heatmaps
print(f"Class 0: Average heatmap calculated from {count_class_0} true predictions.")
print('count_class_0: ',count_class_0)


# Class 0
plt.figure(figsize=(8, 8))

plt.imshow(average_heatmap_class_0,interpolation='bilinear', cmap='viridis',origin='lower', aspect='auto',  extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()])
plt.colorbar()
plt.title('Average Grad-CAM Heatmap for Class 0')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">CLASS 1 PREDICTED 1</span>
</div>

In [ ]:

image_size = (100, 100)
n_patches = model.n_patches
patch_size = image_size[0] // n_patches

# Instantiate Grad-CAM
grad_cam = GradCAM(model)
cam = grad_cam(test_data_tensor, target_class=1)  # Grad-CAM output, shape: (1058, 25)

# Initialize containers for average heatmaps
average_heatmap_class_1 = np.zeros(image_size)
count_class_1 = 0

# Loop through all data points to find true predictions
for idx in range(len(actual_labels)):
    if actual_labels[idx] == predicted_labels[idx]:  # Check if the prediction is correct
        cam_batch = cam[idx, :].detach().cpu().numpy()  # Grad-CAM for the current batch, shape: (25,)
        heatmap = np.zeros(image_size)
        
        # Reconstruct the heatmap from patches
        for i in range(n_patches):
            for j in range(n_patches):
                patch_idx = i * n_patches + j
                patch_value = cam_batch[patch_idx]  # Get Grad-CAM value for this patch
                heatmap[
                    i * patch_size: (i + 1) * patch_size,
                    j * patch_size: (j + 1) * patch_size
                ] = patch_value
        
        # Accumulate heatmaps for the corresponding class
        if actual_labels[idx] == 1:  # True prediction for class 1
            average_heatmap_class_1 += heatmap
            count_class_1 += 1

# Compute average heatmaps
if count_class_1 > 0:
    average_heatmap_class_1 /= count_class_1


# Normalize the average heatmaps (optional, for visualization)
average_heatmap_class_1 -= average_heatmap_class_1.min()
average_heatmap_class_1 /= average_heatmap_class_1.max()



# Final heatmaps
print(f"Class 1: Average heatmap calculated from {count_class_1} true predictions.")
print('count_class_1: ',count_class_1)


# Class 0
plt.figure(figsize=(8, 8))

plt.imshow(average_heatmap_class_1,interpolation='bilinear', cmap='viridis',origin='lower', aspect='auto',  extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()])
plt.colorbar()
plt.title('Average Grad-CAM Heatmap for Class 1')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">CLASS 0 PREDICTED 1</span>
</div>

In [ ]:
cam = grad_cam(test_data_tensor, target_class=1)  
average_heatmap_actual_0_pred_1 = np.zeros(image_size)
count_actual_0_pred_1 = 0

for idx in range(len(actual_labels)):
    if actual_labels[idx] == 0 and predicted_labels[idx] == 1:  # Actual 0, Predicted 1
        cam_batch = cam[idx, :].detach().cpu().numpy()  # Grad-CAM for the current batch, shape: (25,)
        heatmap = np.zeros(image_size)
        
        # Reconstruct the heatmap from patches
        for i in range(n_patches):
            for j in range(n_patches):
                patch_idx = i * n_patches + j
                patch_value = cam_batch[patch_idx]  # Get Grad-CAM value for this patch
                heatmap[
                    i * patch_size: (i + 1) * patch_size,
                    j * patch_size: (j + 1) * patch_size
                ] = patch_value
        
        # Accumulate heatmaps for averaging
        average_heatmap_actual_0_pred_1 += heatmap
        count_actual_0_pred_1 += 1

# Normalize the average heatmap if there were misclassified samples
if count_actual_0_pred_1 > 0:
    average_heatmap_actual_0_pred_1 /= count_actual_0_pred_1
    
average_heatmap_actual_0_pred_1 -= average_heatmap_actual_0_pred_1.min()
average_heatmap_actual_0_pred_1 /= average_heatmap_actual_0_pred_1.max()

# Final heatmaps
print(f"Misclassification: Actual 0, Predicted 1 - {count_actual_0_pred_1} samples.")

# Plotting
plt.figure(figsize=(8, 8))
plt.imshow(average_heatmap_actual_0_pred_1, interpolation='bilinear', cmap='viridis', origin='lower', 
           aspect='auto', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()])
plt.colorbar()
plt.title('Average Grad-CAM: Actual 0, Predicted 1')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()


<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(238, 206, 148),rgb(238, 150, 150)); padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 8px 16px rgba(0,0,0,0.19), 0 4px 4px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 1.2em; color: white; text-shadow: 1px 1px 3px #000;">Class 1 PREDICTED 0</span>
</div>

In [ ]:
cam = grad_cam(test_data_tensor, target_class=0)  
average_heatmap_actual_1_pred_0 = np.zeros(image_size)
count_actual_1_pred_0 = 0

for idx in range(len(actual_labels)):
    if actual_labels[idx] == 1 and predicted_labels[idx] == 0:  # Actual 1, Predicted 0
        cam_batch = cam[idx, :].detach().cpu().numpy() 
        heatmap = np.zeros(image_size)
        
     
        for i in range(n_patches):
            for j in range(n_patches):
                patch_idx = i * n_patches + j
                patch_value = cam_batch[patch_idx]  
                heatmap[
                    i * patch_size: (i + 1) * patch_size,
                    j * patch_size: (j + 1) * patch_size
                ] = patch_value
        
      
        average_heatmap_actual_1_pred_0 += heatmap
        count_actual_1_pred_0 += 1

if count_actual_1_pred_0 > 0:
    average_heatmap_actual_1_pred_0 /= count_actual_1_pred_0
    
average_heatmap_actual_1_pred_0 -= average_heatmap_actual_1_pred_0.min()
average_heatmap_actual_1_pred_0 /= average_heatmap_actual_1_pred_0.max()


print(f"Misclassification: Actual 1, Predicted 0 - {count_actual_1_pred_0} samples.")

# Plotting
plt.figure(figsize=(8, 8))
plt.imshow(average_heatmap_actual_1_pred_0, interpolation='bilinear', cmap='viridis', origin='lower', 
           aspect='auto', extent=[time_axis.min(), time_axis.max(), frequency_axis.min(), frequency_axis.max()])
plt.colorbar()
plt.title('Average Grad-CAM: Actual 1, Predicted 0')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()
